In [188]:
!pip install pyspark



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [189]:
import pyspark
from pyspark.sql import functions as f

In [190]:
import pandas as pd

In [191]:
!pip install pandas


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [192]:
import pandas as pd

In [193]:
pd.read_csv('chicago_crime_data.csv', low_memory=False)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,12.0,61.0,08B,1165074.0,1875917.0,2015.0,02/10/2018 03:50:01 PM,41.815117,-87.670000,"(41.815117282, -87.669999562)"
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,...,29.0,25.0,06,1138875.0,1904869.0,2015.0,02/10/2018 03:50:01 PM,41.895080,-87.765400,"(41.895080471, -87.765400451)"
2,11646166,JC213529,09/01/2018 12:01:00 AM,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,False,True,...,8.0,44.0,06,NaN,NaN,2018.0,04/06/2019 04:04:43 PM,NaN,NaN,NaN
3,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,...,35.0,21.0,18,1152037.0,1920384.0,2015.0,02/10/2018 03:50:01 PM,41.937406,-87.716650,"(41.937405765, -87.716649687)"
4,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,...,28.0,25.0,08A,1141706.0,1900086.0,2015.0,02/10/2018 03:50:01 PM,41.881903,-87.755121,"(41.881903443, -87.755121152)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4071165,5789137,HN594889,09/17/2007 06:00:00 AM,100XX W OHARE ST,0810,THEFT,OVER $500,AIRPORT/AIRCRAFT,False,False,...,41.0,76.0,06,1100658.0,1934241.0,2007.0,02/28/2{,NaN,NaN,NaN
4071166,"""error"" : true",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4071167,"""message"" : ""Internal error""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4071168,"""status"" : 500",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [194]:
from pyspark.sql import SparkSession


In [195]:
def crime_activity_based_on_location(df, location):
    """
    This function returns the criminal activity based on location
    Args:
        df: Spark DataFrame
        location: Location Description
    Returns:
        Spark DataFrame
    """
    return df.filter(df["Location Description"]==location).groupBy("Primary Type").count().orderBy('count', ascending=False)


In [196]:
def crime_count(df):
    """
    This function returns the number of crimes in each category
    Args:
        df: Spark DataFrame
    Returns:
        Spark DataFrame
    """
    return df.groupBy("Primary Type").count().orderBy('count', ascending=False)

In [197]:
# Function where when IUCR is passed, it returns the description of the crime
def crime_description(df, iucr):
    """
    This function returns the description of the crime based on IUCR
    Args:
        df: Spark DataFrame
        iucr: IUCR
    Returns:
        Spark DataFrame
    """
    return df.filter(df["IUCR"]==iucr).select("Primary Type", "Description").distinct()

In [198]:
# Function where when I pass the IUCR code, it returns the percentage of the crime in the dataset
def crime_percentage(df, iucr):
    """
    This function returns the percentage of the crime in the dataset based on IUCR
    Args:
        df: Spark DataFrame
        iucr: IUCR
    Returns:
        float
    """
    return df.filter(df["IUCR"]==iucr).count()/df.count()*100

In [199]:
# create a data frame where first colum is crime name, 2nd column is crime in year1 and 3rd column is crime in year2
def compare_crimes(df, year1, year2):
    """
    This function returns the number of crimes in each category in year1 and year2
    Args:
        df: Spark DataFrame
        year1: Year
        year2: Year
    Returns:
        Spark DataFrame
    """
    df_year1 = df.filter(df["Year"]==year1).groupBy("Primary Type").count().orderBy('count', ascending=False)
    df_year2 = df.filter(df["Year"]==year2).groupBy("Primary Type").count().orderBy('count', ascending=False)
    df_year1 = df_year1.withColumnRenamed("count", str(year1))
    df_year2 = df_year2.withColumnRenamed("count", str(year2))
    df_year1 = df_year1.withColumnRenamed("Primary Type", "Primary Type1")
    df_year2 = df_year2.withColumnRenamed("Primary Type", "Primary Type2")
    df_year1 = df_year1.withColumn("Primary Type", df_year1["Primary Type1"])
    df_year2 = df_year2.withColumn("Primary Type", df_year2["Primary Type2"])
    df_year1 = df_year1.drop("Primary Type1")
    df_year2 = df_year2.drop("Primary Type2")
    df_year1 = df_year1.join(df_year2, on="Primary Type", how="outer")
    return df_year1
  

In [200]:
def most_happening_crime(df, year):
    """
    This function returns the most happening crime based on IUCR code in a year
    Args:
        df: Spark DataFrame
        year: Year
    Returns:
        Spark DataFrame
    """
    return df.filter(df["Year"]==year).groupBy("Primary Type").count().orderBy('count', ascending=False)

In [ ]:
# total number of crimes in Chicago
def total_crimes(df):
    """
    This function returns the total number of crimes in Chicago
    Args:
        df: Spark DataFrame
    Returns:
        Spark DataFrame
    """
    return df.count()

In [201]:
def count_crimes_in_block(df, crime, block):
    """
    This function returns the count of crimes in a block
    Args:
        df: Spark DataFrame
        crime: Crime
        block: Block
    Returns:
        Spark DataFrame
    """
    return df.filter(df["Primary Type"]==crime).filter(df["Block"]==block).groupBy("Primary Type").count().orderBy('count', ascending=False)

In [ ]:
import datetime
def convert_year(year):
    """
    This function converts the year from bigint to timestamp
    Args:
        year: Year
    Returns:
        year: Year
    """
    return datetime.datetime(year, 1, 1)
# convert the year from bigint to timestamp
dataframe_crime_year_by_year=crime_year_by_year(df_pyspark)
dataframe_crime_year_by_year=dataframe_crime_year_by_year.withColumn("Year", dataframe_crime_year_by_year["Year"].cast("timestamp"))
dataframe_crime_year_by_year.write.csv('crime_year_by_year.csv', header='True')


In [205]:
def common_crimes_in_block(df, crime):
    """
    This function returns the common crimes in a block
    Args:
        df: Spark DataFrame
        crime: Crime
    Returns:
        Spark DataFrame
    """
    return df.filter(df["Primary Type"]==crime).groupBy("Block").count().orderBy('count', ascending=False)

In [ ]:
# Crimes on each day 
def crime_on_each_day(df):
    """
    This function returns the number of crimes on each day
    Args:
        df: Spark DataFrame
    Returns:
        Spark DataFrame
    """
    return df.groupBy("Date").count().orderBy('count', ascending=False)


In [206]:
def most_crimes_occuring_block(df):
  """
  This function returns the most crimes occuring block in Chicago
  Args:
      df: Spark DataFrame
  Returns:
      Spark DataFrame
  """
  return df.groupBy("Block").count().orderBy('count', ascending=False)

In [207]:
def crime_year_by_year(df):
  """
  This function returns the year with the highest crime
  Args:
      df: Spark DataFrame
  Returns:
      Spark DataFrame
  """
  return df.groupBy("Year").count().orderBy('count', ascending=False)
  

In [243]:
def crime_location(df, crime):
    """
    This function returns the latitude and longitude of the crime that happened the most limit it to top 20
    Args:
        df: Spark DataFrame
        crime: Crime
    Returns:
        Spark DataFrame
    """
    return df.filter(df["Primary Type"]==crime).groupBy("Latitude", "Longitude").count().orderBy('count', ascending=False).limit(20)

In [209]:
# time at which particular crime is most likely to happen
def crime_time(df, crime):
    """
    This function returns the time at which particular crime is most likely to happen
    Args:
        df: Spark DataFrame
        crime: Crime
    Returns:
        Spark DataFrame
    """
    return df.filter(df["Primary Type"]==crime).groupBy("Time").count().orderBy('count', ascending=False)

In [210]:
from datetime import datetime
def convert_time(time):
    """
    This function converts the time from 12 hour format to 24 hour format
    Args:
        time: Time
    Returns:
        time: Time
    """
    input_format = '%m/%d/%Y %I:%M:%S %p'
    output_format = '%H:%M:%S'
    time = datetime.strptime(time, input_format)
    time = time.strftime(output_format)
    return time

time_udf = f.udf(lambda Time:convert_time(Time))
df_pyspark_date = df_pyspark.withColumn("Time", time_udf(df_pyspark["Date"]))
df_pyspark_date.show()

# Extract date from Date column
def extract_date(date):
    """
    This function extracts the date from Date column
    Args:
        date: Date
    Returns:
        date: Date
    """
    input_format = '%m/%d/%Y %I:%M:%S %p'
    output_format = '%m/%d/%Y'
    date = datetime.strptime(date, input_format)
    date = date.strftime(output_format)
    return date

date_udf = f.udf(lambda Date:extract_date(Date))
df_pyspark_date = df_pyspark_date.withColumn("Date", date_udf(df_pyspark_date["Date"]))
df_pyspark_date.show()


+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+--------+
|      ID|Case Number|                Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|    Time|
+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+--------+
|10224738|   HY411648|09/05/2015 01:30:...|     043XX S WOOD ST|0486|           BATTERY|DOMESTIC BATTER

+--------+-----------+----------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+--------+
|      ID|Case Number|      Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|    Time|
+--------+-----------+----------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+--------+
|10224738|   HY411648|09/05/2015|     043XX S WOOD ST|0486|           BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    tr

In [ ]:
# Reduction in crime in a block from 2002 to 2022
def reduction_in_crime(df, block):
    """
    This function returns the reduction in crime in a block from 2002 to 2022
    Args:
        df: Spark DataFrame
        block: Block
    Returns:
        Spark DataFrame
    """
    return df.filter(df["Block"]==block).groupBy("Year").count().orderBy('Year', ascending=True)

In [ ]:
def total_arrest(df):
    """
    This function returns the total arrest in every year
    Args:
        df: Spark DataFrame
    Returns:
        Spark DataFrame
    """
    return df.filter(df["Arrest"]==True).groupBy("Year").count().orderBy('Year', ascending=True)

In [ ]:
def percentage_arrest(totalCrimes, totalArrest):
    """
    This function returns the percentage of arrest for a particular crime
    Args:
        totalCrimes: Total number of crimes
        totalArrest: Total number of arrests
    Returns:
        percentage: Percentage of arrest
    """
    percentage = (totalArrest/totalCrimes)*100
    return percentage

In [ ]:
def most_common_crime_in_block(df, block):
    """
    This function returns the most common crime in a particular block
    Args:
        df: Spark DataFrame
        block: Block
    Returns:
        Spark DataFrame
    """
    return df.filter(df["Block"]==block).groupBy("Primary Type").count().orderBy('count', ascending=False)

In [ ]:
def crime_percentage_arrest(df, totalCrimes, totalArrest):
    """
    This function returns the percentage of arrest for a particular crime
    Args:
        df: Spark DataFrame
        totalCrimes: Total number of crimes
        totalArrest: Total number of arrests
    Returns:
        Spark DataFrame
    """
    return df.withColumn("Percentage Arrest", percentage_arrest(totalCrimes, totalArrest))
    

In [ ]:
spark= SparkSession.builder.appName('Peoject').getOrCreate()

23/02/15 11:42:03 WARN Utils: Your hostname, RACs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.190.229.59 instead (on interface en0)
23/02/15 11:42:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/15 11:42:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
df_pyspark= spark.read.csv('chicago_crime_data.csv', header='True')

In [ ]:
df_pyspark.show()

+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|09/05/2015 01:30:...|     043XX S WOOD ST|0486|           BATTERY|DOMESTIC BATTERY ...|           RESIDENCE|

In [ ]:
spark.read.option('header', 'true').csv('chicago_crime_data.csv')


DataFrame[ID: string, Case Number: string, Date: string, Block: string, IUCR: string, Primary Type: string, Description: string, Location Description: string, Arrest: string, Domestic: string, Beat: string, District: string, Ward: string, Community Area: string, FBI Code: string, X Coordinate: string, Y Coordinate: string, Year: string, Updated On: string, Latitude: string, Longitude: string, Location: string]

In [ ]:
df_pyspark.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



In [ ]:
df_pyspark.filter("Beat==1424").show()

+--------+-----------+--------------------+--------------------+----+-------------------+-------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|               Block|IUCR|       Primary Type|        Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+--------------------+----+-------------------+-------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224752|   HY411601|09/03/2015 01:00:...| 020XX W SCHILLER ST|0810|              THEFT|          OVER $500|              STREET|

In [ ]:
df_pyspark.filter(df_pyspark["Primary Type"]=="THEFT").show()

+--------+-----------+--------------------+--------------------+----+------------+--------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|               Block|IUCR|Primary Type|   Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+--------------------+----+------------+--------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224739|   HY411615|09/04/2015 11:30:...| 008XX N CENTRAL AVE|0870|       THEFT|POCKET-PICKING|             CTA BUS| false|   false|1511|     015|  29|            2

In [ ]:
# We have grouped the data by Primary Type and counted the number of crimes in each category in descending order
df_pyspark.groupBy("Primary Type").count().orderBy('count', ascending=False).show()

+--------------------+------+
|        Primary Type| count|
+--------------------+------+
|               THEFT|837270|
|             BATTERY|758486|
|     CRIMINAL DAMAGE|477924|
|           NARCOTICS|407897|
|       OTHER OFFENSE|263480|
|             ASSAULT|259499|
|            BURGLARY|214695|
| MOTOR VEHICLE THEFT|193902|
|  DECEPTIVE PRACTICE|165308|
|             ROBBERY|151617|
|   CRIMINAL TRESPASS|123519|
|   WEAPONS VIOLATION| 46416|
|        PROSTITUTION| 45112|
|OFFENSE INVOLVING...| 26634|
|PUBLIC PEACE VIOL...| 24549|
|         SEX OFFENSE| 16258|
| CRIM SEXUAL ASSAULT| 15605|
|LIQUOR LAW VIOLATION|  9020|
|INTERFERENCE WITH...|  8299|
|            GAMBLING|  7866|
+--------------------+------+
only showing top 20 rows



In [ ]:
df_pyspark.agg({"Primary Type": "count"}).show()

+-------------------+
|count(Primary Type)|
+-------------------+
|            4071166|
+-------------------+



In [ ]:
# Apply the function to the dataframe
crime_count(df_pyspark).show()

+--------------------+------+
|        Primary Type| count|
+--------------------+------+
|               THEFT|837270|
|             BATTERY|758486|
|     CRIMINAL DAMAGE|477924|
|           NARCOTICS|407897|
|       OTHER OFFENSE|263480|
|             ASSAULT|259499|
|            BURGLARY|214695|
| MOTOR VEHICLE THEFT|193902|
|  DECEPTIVE PRACTICE|165308|
|             ROBBERY|151617|
|   CRIMINAL TRESPASS|123519|
|   WEAPONS VIOLATION| 46416|
|        PROSTITUTION| 45112|
|OFFENSE INVOLVING...| 26634|
|PUBLIC PEACE VIOL...| 24549|
|         SEX OFFENSE| 16258|
| CRIM SEXUAL ASSAULT| 15605|
|LIQUOR LAW VIOLATION|  9020|
|INTERFERENCE WITH...|  8299|
|            GAMBLING|  7866|
+--------------------+------+
only showing top 20 rows



In [ ]:
# Applied the function to the dataframe
crime_description(df_pyspark, "1153")

+------------------+--------------------+
|      Primary Type|         Description|
+------------------+--------------------+
|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|
+------------------+--------------------+



In [ ]:
crime_percentage(df_pyspark, "1153")

0.5171240699847955

In [ ]:
# Crimes compared for 2002 and 2020
compare_crimes(df_pyspark, 2002, 2020)
percentage_change_in_crime(df_pyspark, 2002, 2020)

+--------------------+-----+
|        Primary Type|count|
+--------------------+-----+
|               THEFT|91806|
|             BATTERY|88041|
|     CRIMINAL DAMAGE|53987|
|           NARCOTICS|49324|
|       OTHER OFFENSE|31531|
|             ASSAULT|29219|
|            BURGLARY|24867|
| MOTOR VEHICLE THEFT|23145|
|             ROBBERY|17433|
|   CRIMINAL TRESPASS|13409|
|  DECEPTIVE PRACTICE|13137|
|        PROSTITUTION| 6007|
|   WEAPONS VIOLATION| 4136|
|OFFENSE INVOLVING...| 2387|
|PUBLIC PEACE VIOL...| 2347|
|         SEX OFFENSE| 2038|
| CRIM SEXUAL ASSAULT| 1675|
|LIQUOR LAW VIOLATION| 1359|
|               ARSON|  938|
|            GAMBLING|  912|
+--------------------+-----+
only showing top 20 rows



+--------------------+-----+
|        Primary Type|count|
+--------------------+-----+
|             BATTERY| 3836|
|               THEFT| 3338|
|     CRIMINAL DAMAGE| 2213|
|             ASSAULT| 1583|
|  DECEPTIVE PRACTICE| 1206|
|       OTHER OFFENSE| 1006|
|            BURGLARY|  900|
| MOTOR VEHICLE THEFT|  846|
|   WEAPONS VIOLATION|  755|
|             ROBBERY|  674|
|           NARCOTICS|  405|
|   CRIMINAL TRESPASS|  305|
|OFFENSE INVOLVING...|  101|
|CRIMINAL SEXUAL A...|   77|
|PUBLIC PEACE VIOL...|   70|
|         SEX OFFENSE|   67|
|INTERFERENCE WITH...|   44|
|               ARSON|   34|
|            STALKING|   22|
|LIQUOR LAW VIOLATION|   10|
+--------------------+-----+
only showing top 20 rows



-96.18627188485675

In [ ]:
crime_activity_based_on_location(df_pyspark, "STREET")

+--------------------+------+
|        Primary Type| count|
+--------------------+------+
|               THEFT|226378|
|     CRIMINAL DAMAGE|170464|
|           NARCOTICS|160478|
| MOTOR VEHICLE THEFT|150731|
|             BATTERY|140674|
|             ASSAULT| 54829|
|             ROBBERY| 51699|
|       OTHER OFFENSE| 47223|
|        PROSTITUTION| 38161|
|   WEAPONS VIOLATION| 15842|
|  DECEPTIVE PRACTICE| 12314|
|PUBLIC PEACE VIOL...|  6905|
|   CRIMINAL TRESPASS|  6161|
|INTERFERENCE WITH...|  4193|
|LIQUOR LAW VIOLATION|  2076|
|         SEX OFFENSE|  2017|
|            GAMBLING|  1365|
|          KIDNAPPING|  1307|
|OFFENSE INVOLVING...|  1208|
| CRIM SEXUAL ASSAULT|   792|
+--------------------+------+
only showing top 20 rows



In [ ]:
pd.read_csv('Chicago_iucr-codes.csv', low_memory=False)

,IUCR,PRIMARY DESCRIPTION,SECONDARY DESCRIPTION,INDEX CODE,ACTIVE
0,110,HOMICIDE,FIRST DEGREE MURDER,I,True
1,130,HOMICIDE,SECOND DEGREE MURDER,I,True
2,141,HOMICIDE,INVOLUNTARY MANSLAUGHTER,N,True
3,142,HOMICIDE,RECKLESS HOMICIDE,N,True
4,261,CRIMINAL SEXUAL ASSAULT,AGGRAVATED - HANDGUN,I,True
...,...,...,...,...,...
405,5121,OTHER OFFENSE,ARSONIST - ANNUAL REGISTRATION,N,True
406,5122,OTHER OFFENSE,ARSONIST - FAIL TO REGISTER NEW ADDRESS,N,True
407,5130,OTHER OFFENSE,VIOLENT OFFENDER - DUTY TO REGISTER,N,True
408,5131,OTHER OFFENSE,VIOLENT OFFENDER - ANNUAL REGISTRATION,N,True


In [ ]:
# Create a new dataframe with the IUCR codes and descriptions
df_iucr = spark.read.option('header', 'true').csv('Chicago_iucr-codes.csv')

In [ ]:
most_happening_crime(df_pyspark, 2009)

+--------------------+-----+
|        Primary Type|count|
+--------------------+-----+
|  DECEPTIVE PRACTICE|   56|
|OFFENSE INVOLVING...|   31|
| CRIM SEXUAL ASSAULT|   15|
|         SEX OFFENSE|    8|
|               THEFT|    7|
|            BURGLARY|    5|
|           NARCOTICS|    2|
|             BATTERY|    1|
|             ASSAULT|    1|
|             ROBBERY|    1|
+--------------------+-----+



In [ ]:
count_crimes_in_block(df_pyspark, "THEFT", "0000X W TERMINAL ST")

+------------+-----+
|Primary Type|count|
+------------+-----+
|       THEFT|  830|
+------------+-----+



In [ ]:
d1=common_crimes_in_block(df_pyspark, "ROBBERY")

+--------------------+-----+
|               Block|count|
+--------------------+-----+
| 063XX S ASHLAND AVE|  154|
| 079XX S ASHLAND AVE|  137|
|  040XX W MADISON ST|  135|
|     0000X W 69TH ST|  127|
|063XX S DR MARTIN...|  126|
|     0000X W 95TH ST|  121|
|079XX S COTTAGE G...|  121|
|  079XX S HALSTED ST|  121|
|     0000X W 79TH ST|  118|
|     001XX W 87TH ST|  112|
|  005XX S PULASKI RD|  106|
|    046XX N BROADWAY|  103|
|  039XX W MADISON ST|  102|
|    022XX S STATE ST|   99|
| 062XX S ASHLAND AVE|   99|
|  066XX S HALSTED ST|   98|
|002XX W GARFIELD ...|   94|
|038XX W ROOSEVELT RD|   93|
| 003XX N CENTRAL AVE|   92|
|     002XX W 63RD ST|   92|
+--------------------+-----+
only showing top 20 rows



In [ ]:
dataframe_crime_activity_based_on_location=crime_activity_based_on_location(df_pyspark, "STREET")
dataframe_crime_activity_based_on_location.write.csv('crime_activity_based_on_location.csv')

In [ ]:
compare_crimes(df_pyspark,2002, 2010)

+--------------------+-----+
|        Primary Type|count|
+--------------------+-----+
|               THEFT|91806|
|             BATTERY|88041|
|     CRIMINAL DAMAGE|53987|
|           NARCOTICS|49324|
|       OTHER OFFENSE|31531|
|             ASSAULT|29219|
|            BURGLARY|24867|
| MOTOR VEHICLE THEFT|23145|
|             ROBBERY|17433|
|   CRIMINAL TRESPASS|13409|
|  DECEPTIVE PRACTICE|13137|
|        PROSTITUTION| 6007|
|   WEAPONS VIOLATION| 4136|
|OFFENSE INVOLVING...| 2387|
|PUBLIC PEACE VIOL...| 2347|
|         SEX OFFENSE| 2038|
| CRIM SEXUAL ASSAULT| 1675|
|LIQUOR LAW VIOLATION| 1359|
|               ARSON|  938|
|            GAMBLING|  912|
+--------------------+-----+
only showing top 20 rows



+--------------------+-----+
|        Primary Type|count|
+--------------------+-----+
|  DECEPTIVE PRACTICE|  126|
|OFFENSE INVOLVING...|   41|
| CRIM SEXUAL ASSAULT|   19|
|         SEX OFFENSE|   14|
|               THEFT|    8|
|             BATTERY|    5|
|           NARCOTICS|    4|
|             ASSAULT|    2|
|             ROBBERY|    2|
|CRIMINAL SEXUAL A...|    1|
|           OBSCENITY|    1|
|   HUMAN TRAFFICKING|    1|
+--------------------+-----+



(None, None)

In [ ]:
# df_pyspark_date.write.csv('chicago_crime_data.csv', header='True')

In [ ]:
print(crime_location(df_pyspark, "THEFT"))

In [ ]:
dataframe_crime_count=crime_count(df_pyspark)
dataframe_crime_count.write.csv('crime_count.csv', header='True')


In [244]:
dataframe_crime_latitude_longitude=crime_location(df_pyspark, "THEFT")
dataframe_crime_latitude_longitude.write.csv('crime_latitude_longitude.csv', header='True')

In [ ]:
dataframe_most_crimes_occuring_block=most_crimes_occuring_block(df_pyspark)
dataframe_most_crimes_occuring_block.write.csv('most_crimes_occuring_block.csv', header='True')

In [ ]:
# # crime_time(df_pyspark_date, "THEFT").show()
# # taking too much time
# dataframe_crime_time=crime_time(df_pyspark_date, "THEFT")
# dataframe_crime_time.write.csv('crime_time.csv', header='True')

In [ ]:
# Crimes compared for 2002 and 2020
dataframe_compare_crimes=compare_crimes(df_pyspark, 2002, 2020)
dataframe_compare_crimes.write.csv('compare_crimes.csv', header='True')

In [212]:
dataframe_most_common_crime_in_block=most_common_crime_in_block(df_pyspark, "100XX W OHARE ST")
dataframe_most_common_crime_in_block.write.csv('most_common_crime_in_block.csv', header='True')


In [217]:
dataframe_total_arrest=total_arrest(df_pyspark)
dataframe_total_arrest.write.csv('total_arrest.csv', header='True')

In [222]:
dataframe_reduction_in_crime=reduction_in_crime(df_pyspark, "100XX W OHARE ST")
dataframe_reduction_in_crime.write.csv('reduction_in_crime.csv', header='True')